In [4]:
import lateral_signaling as lsig
import numpy as np
import pandas as pd
from tqdm import tqdm
import numba

import scipy.stats as st
from scipy.sparse import csr_matrix

import os
from glob import glob

In [5]:
def tc_rhs_beta_no_g(S, S_delay, Adj, sender_idx, beta_func, beta_args, alpha, k, p, delta, lambda_, rho):
    """
    Right-hand side of the transciever circuit delay 
    differential equation. Uses a matrix of cell-cell contact 
    lengths `L`.
    """

    # Get signaling as a function of density
    beta = beta_func(rho, *beta_args)
    
    # Get input signal across each interface
    S_bar = beta * (Adj @ S_delay)

    # Calculate dE/dt
    dS_dt = (
        lambda_
        + alpha
        * (S_bar ** p)
        / (
            k ** p 
            + (delta * S_delay) ** p 
            + S_bar ** p
        )
        - S
    )

    # Set sender cell to zero
    dS_dt[sender_idx] = 0

    return dS_dt


def tc_rhs_beta_g_normA(S, S_delay, Adj, sender_idx, beta_func, beta_args, alpha, k, p, delta, lambda_, g, rho):
    """
    Right-hand side of the transciever circuit delay 
    differential equation. Uses a matrix of cell-cell contact 
    lengths `L`.
    """

    # Get signaling as a function of density
    beta = beta_func(rho, *beta_args)
    
    # Get input signal across each interface
    S_bar = beta * (Adj @ S_delay)

    # Calculate dE/dt
    dS_dt = (
        lambda_
        + alpha
        * (S_bar ** p)
        / (
            ((g ** 2) * k) ** p 
            + (delta * S_delay) ** p 
            + S_bar ** p
        )
        - g * S
    )

    # Set sender cell to zero
    dS_dt[sender_idx] = 0

    return dS_dt


# In[3]:


from scipy.sparse import csr_matrix, diags, identity
import scipy.stats

def k_step_Adj(k, rows, cols=0, dtype=np.float32, row_stoch=False, **kwargs):
    """
    """
    
    if not cols:
        cols = rows
        
    # Construct adjacency matrix
    a = lsig.make_Adj_sparse(rows, cols, dtype=dtype, **kwargs)
    
    # Add self-edges
    n = rows * cols
    eye = identity(n).astype(dtype)
    A = (a + eye)
    
    # Compute number of paths of length k between nodes
    A = A ** k
    
    # Store as 0. or 1.
    A = (A > 0).astype(dtype)
    
    # Remove self-edges
    A = A - diags(A.diagonal())
    
    if row_stoch:
        rowsum = np.sum(A, axis=1)
        A = csr_matrix(A / rowsum)
    
    return A


# <hr>

# In[27]:

In [18]:
# Set directory
# wd = "/home/pbhamidi/git/evomorph/lateral_signaling/"
wd = "C:/Users/Pranav/git/evomorph/lateral_signaling/"
os.chdir(wd)
print("Current directory:", os.getcwd())

# Directory to save results
res_dir = os.path.realpath(os.path.join("..", "data"))
assert os.path.exists(res_dir), f"Directory does not exist: {res_dir}"
print("Will save to directory:", res_dir)

Current directory: C:\Users\Pranav\git\evomorph\lateral_signaling
Will save to directory: C:\Users\Pranav\git\evomorph\data


In [7]:
# Unique name of current run
run_name = "20210707_sweep_TCphase"

# Directory of parameter set
trial_dir = "."

# Name of parameter set
trial_name = "lowcis_expbeta"

# Set random seed
seed = 2021
np.random.seed(seed)

# Whether to display progress
progress_bar = True

# Set floating-point tolerance
tol = 1e-5

# __Set growth min and max__

# In[5]:


# Set min and max density
rho_min, rho_max = 1, 5.63040245


# __Set RHS of dynamical equation__

# Set the RHS function in long-form
rhs_long = tc_rhs_beta_no_g

# Set beta(rho)
beta_func = lsig.beta_rho_exp

In [44]:
# Search trial_dir for parameter set CSV
params_regexp = "*" + trial_name + "*.csv"
params_path = glob(os.path.join(trial_dir, params_regexp))

# Load parameter set
if len(params_path) == 0:
    raise FileNotFoundError(
        f"No file matches the regular expression `{params_regexp}` "
        + f"in the directory `{os.path.abspath(trial_dir)}` ."
    )
elif len(params_path) > 1:
    raise FileNotFoundError(
        f"More than one file matches the regular expression "
        + f"`{params_regexp}` in the directory "
        + f"`{os.path.abspath(trial_dir)}` "
    )
else:
    # Read trial parameters
    params_path = os.path.abspath(params_path[0])
    params_df = pd.read_csv(params_path)


# In[10]:


# Get any arguments for beta function
is_beta_arg = [p.startswith("beta_") for p in params_df["parameter"].values]
beta_args   = params_df.value.values[is_beta_arg]

# Get the delay parameter
is_delay    = [p == "delay" for p in params_df["parameter"].values]
delay       = params_df.value.values[is_delay][0]

# Package all other parameters 
is_param    = [not (ba or d) for ba, d in zip(is_beta_arg, is_delay)]
param_names = params_df.parameter.values[is_param].astype(str)
param_vals  = params_df.value.values[is_param].astype(np.float32)

# Package arguments for lsig.integrate_DDE and 
#   lsig.integrate_DDE_varargs. Density param is 
#   initialized with rho_min.
dde_args = *param_vals, rho_min

# In[11]:

# Get `g`
where_g = next(i for i, pn in enumerate(param_names) if "g" == pn)
g = param_vals[where_g]

# (Optional) Remove `g` from signaling parameters
dde_args = [*dde_args[:where_g], *dde_args[(where_g+1):]]

# Get index of `rho` (last argument)
where_rho = len(dde_args) - 1

# Get `k`
where_k = next(i for i, pn in enumerate(param_names) if "k" == pn)
k = param_vals[where_k]
thresh = k


# Figure parameters
pct_s = 1

# Set time parameters
tmax = 5
nt_t = 100

# Get time points
nt = int(nt_t * tmax) + 1
t = np.linspace(0, tmax, nt)


# __Construct lattice of cells__

# Make lattice
rows = cols = 100
X = lsig.hex_grid(rows, cols)

# Get # cells
n = X.shape[0]

# Calculate cell adjacency
kAdj_1 = k_step_Adj(1, rows, cols, row_stoch=True)

# Specify percent of population that is sender
n_s = int(n * (pct_s / 100)) + 1

# Define free parameters for parameter scan
rep_space     = np.arange(5)
g_space       = np.linspace(0.25, 2.25, 25)
rho_0_space   = np.linspace(0, rho_max, 25)[1:]
# rho_max_space = np.linspace(0, 8, 25)[1:]
free_params   = (
    rep_space, 
    g_space, 
    rho_0_space, 
#     rho_max_space,
)
free_param_names   = (
    "rep", 
    "g", 
    "rho_0", 
#     "rho_max",
)

# Make array with all combinations of params
param_space = np.meshgrid(*free_params)
param_space = np.array(param_space).T.reshape(-1, len(free_params))

# Get sender indices for each replicate
sender_idx_rep = np.empty((rep_space.size, n_s), dtype=int)
for rep in rep_space:    
    # Set random seed
    np.random.seed(seed + rep)
    # Assign senders randomly
    sender_idx = np.random.choice(n, n_s, replace=False)
    sender_idx_rep[rep] = sender_idx

# Get number of simulations
n_runs = param_space.shape[0]

In [33]:
def rhs_reporter(R, R_delay, S, gamma_R):
    dR_dt = (S - R) * gamma_R
    
    dR_dt[sender_idx] = 0
    
    return dR_dt

# Set parameter(s)
gamma_R = 0.1
R_args = [S_t[0], gamma_R]

# Set initial expression
R0 = np.zeros(n, dtype=np.float32)

In [13]:
# Make mutable copy of dde args
args = list(dde_args)

# Initialize results vectors
S_actnum_param = np.empty((n_runs, nt), dtype=int)
S_tcmean_param = np.empty((n_runs, nt), dtype=np.float32)
# R_actnum_param = np.empty((n_runs, nt), dtype=int)
# R_tcmean_param = np.empty((n_runs, nt), dtype=np.float32)

# Make iterator
iterator = range(n_runs)

# # Make test iterator
# iterator = range(24)

if progress_bar:
    iterator = tqdm(iterator)

for i in iterator:    
    
    # Get parameters
    rep, g_, rho_0_ = param_space[i]
    rho_max_ = rho_max
#     rep, g_, rho_0_, rho_max_ = param_space[i]
#     args[where_g] = g_
    
    # Get senders
    sender_idx = sender_idx_rep[int(rep)]
    
    # Make a mask for transceivers
    tc_mask = np.ones(n, dtype=bool)
    tc_mask[sender_idx] = False
    
    # Get RHS of DDE equation to pass to integrator
    rhs_1 = lsig.get_DDE_rhs(rhs_long, kAdj_1, sender_idx, beta_func, beta_args,)
    
    # Get initial conditions
    S0 = np.zeros(n)
    S0[sender_idx] = 1
    
    # Calculate density
    rho_t = lsig.logistic(t, g_, rho_0_, rho_max_)
    
    # Simulate
    S_t = lsig.integrate_DDE_varargs(
        t,
        rhs_1,
        var_vals=[rho_t],
        where_vars=where_rho,
        dde_args=args,
        E0=S0,
        delay=delay,
        varargs_type="list",
    )
    
    # Number of activated cells
    S_act_t = S_t > thresh
    S_actnum_t = S_act_t.sum(axis=1)
    
    # Mean fluorescence
    S_tcmean_t = S_t[:, tc_mask].mean(axis=1)
    
    # Save results
    S_actnum_param[i] = S_actnum_t
    S_tcmean_param[i] = S_tcmean_t

    # Simulate reporter expression
    R_t = lsig.integrate_DDE_varargs(
        t,
        reporter_rhs,
        var_vals=[S_t],
        where_vars=0,
        dde_args=R_args,
        E0=R0,
        delay=0,
        progress_bar=True,
        min_delay=0,
        varargs_type="list",
    )

    # Number of activated cells
    R_act_t = R_t > thresh
    R_actnum_t = R_act_t.sum(axis=1)
    
    # Mean fluorescence
    R_tcmean_t = R_t[:, tc_mask].mean(axis=1)
    
    # Save results
    R_actnum_param[i] = R_actnum_t
    R_tcmean_param[i] = R_tcmean_t
    
    # Save results for reporter
    R_actnum_param[i] = R_actnum_t
    R_tcmean_param[i] = R_tcmean_t


100%|██████████| 3000/3000 [06:58<00:00,  7.17it/s]


In [52]:
# Store results 
data_dict = dict(
    n=n,
    t=t,
    trial_name       = trial_name,
    param_names      = param_names,
    param_vals       = param_vals,
    beta_args        = beta_args,
    delay            = delay,
    irad             = 1.,
    rho_max          = rho_max,
    random_seeds     = seed+rep_space,
    sender_idx_rep   = sender_idx_rep,
    free_param_names = free_param_names,
    param_space      = param_space,
    S_actnum_param   = S_actnum_param,
    S_tcmean_param   = S_tcmean_param,
    gamma_R          = gamma_R,
    R_actnum_param   = R_actnum_param,
    R_tcmean_param   = R_tcmean_param,
)

# Make results directory if it doesn't exist
data_dir = os.path.join(res_dir, run_name)
if not os.path.exists(data_dir):
    print("Creating directory", data_dir)
    os.mkdir(data_dir)


In [53]:
# Save results 
data_file = os.path.join(
    res_dir, run_name, run_name + "_results"
)

In [54]:
np.savez_compressed(data_file, **data_dict)
print("Mission complete.")

Mission complete.


<hr>